In [1]:
channels = [
    'Leyueqa',
    'nevacomputer',
    'ZemenExpress',
    'meneshayeofficial',
    'ethio_brand_collection'
]


In [2]:
from telethon import TelegramClient
import asyncio
import json
import os

api_id = 20626650
api_hash = '9a7350ab48f6c32d22418085adafd86b'
session_name = 'ethio_session'
phone = '+251954504193'  # remove spaces

channels = [
    'Leyueqa',
    'nevacomputer',
    'ZemenExpress',
    'meneshayeofficial',
    'ethio_brand_collection'
]

client = TelegramClient(session_name, api_id, api_hash)

os.makedirs('data', exist_ok=True)

async def fetch_messages(channel_username, limit=100):
    channel = await client.get_entity(channel_username)
    messages = []
    async for message in client.iter_messages(channel, limit=limit):
        msg_data = {
            "channel": channel_username,
            "sender_id": message.sender_id,
            "date": message.date.isoformat(),
            "text": message.message,
            "media": None
        }
        if message.media:
            file_path = f"data/{channel_username}_{message.id}"
            await message.download_media(file_path)
            msg_data["media"] = file_path
        messages.append(msg_data)

    with open(f'data/{channel_username}_messages.json', 'w', encoding='utf-8') as f:
        json.dump(messages, f, ensure_ascii=False, indent=4)

async def main():
    # Start the client only once here, passing your phone number
    await client.start(phone=phone)
    tasks = [fetch_messages(channel) for channel in channels]
    await asyncio.gather(*tasks)
    await client.disconnect()  # optionally disconnect after done

await main()



OperationalError: database is locked

Server closed the connection: [WinError 10054] An existing connection was forcibly closed by the remote host


In [9]:
from telethon import TelegramClient
import asyncio
import json
import os
import pandas as pd
import re
from datetime import datetime

# === Telegram API credentials ===
api_id = 20626650
api_hash = '9a7350ab48f6c32d22418085adafd86b'
session_name = 'ethio_session'
phone = '+251954504193'  # remove spaces


# === List of e-commerce Telegram channels ===
channels = [
    'Leyueqa',
    'nevacomputer',
    'ZemenExpress',
    'meneshayeofficial',
    'ethio_brand_collection'
]

# === Directory setup ===
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/clean', exist_ok=True)

# === Amharic text preprocessing function ===
def preprocess_amharic(text):
    if not text:
        return ""
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"\s+", " ", text).strip()  # Normalize whitespace
    text = text.replace("\u1369", "1")  # Ethiopic digit example
    return text

# === Fetch messages function ===
async def fetch_messages(channel):
    channel_entity = await client.get_entity(channel)
    messages = []
    async for message in client.iter_messages(channel_entity, limit=100):
        all_records = []
        for channel in channels:
            print(f"Fetching from: {channel}")
            try:
                entity = await client.get_entity(channel)
                async for msg in client.iter_messages(entity, limit=100):
                    record = {
                        'channel': channel,
                        'message_id': msg.id,
                        'date': msg.date.isoformat(),
                        'sender_id': msg.sender_id,
                        'raw_text': msg.message,
                        'clean_text': preprocess_amharic(msg.message),
                        'media': None
                    }
                    # Download media if available
                    if msg.media:
                        media_path = f"data/raw/{channel}_{msg.id}"
                        await msg.download_media(media_path)
                        record['media'] = media_path
                    all_records.append(record)
            except Exception as e:
                print(f"Failed for {channel}: {e}")

        # Save to CSV
        df = pd.DataFrame(all_records)
        df.to_csv("data/clean/preprocessed_messages.csv", index=False, encoding="utf-8-sig")
        print("✅ Data ingestion and preprocessing complete.")

# === Run it ===
if __name__ == "__main__":
    await main()


BotMethodInvalidError: The API access for bot users is restricted. The method you tried to invoke cannot be executed as a bot (caused by GetHistoryRequest)